# FAQs

## How to...

### ... get data for another symbol?

In [1]:
from pybroker import ExecContext, Strategy, YFinance, highest

def exec_fn(ctx: ExecContext):
    if ctx.symbol == 'NVDA':
        other_bar_data = ctx.foreign('AMD')
        other_highest = ctx.indicator('high_10d', 'AMD')
        
strategy = Strategy(YFinance(), start_date='1/1/2022', end_date='1/1/2023')
strategy.add_execution(
   exec_fn, ['NVDA', 'AMD'], indicators=highest('high_10d', 'close', period=10))
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2022-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  2 of 2 completed
Loaded bar data: 0:00:00 

Computing indicators...


100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2022-01-03 00:00:00 to 2022-12-30 00:00:00


100% (251 of 251) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


You can also retrieve models, predictions, and other data for other symbols. [For more information, refer to the ExecContext reference documentation.](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext)

### ... place an order more than one bar ahead?

Use the [buy_delay](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.buy_delay) and [sell_delay](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.sell_delay) configuration options:

In [2]:
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

def buy_fn(ctx: ExecContext):
    if not tuple(ctx.pending_orders()) and not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1

config = StrategyConfig(buy_delay=5)
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023', config=config)
strategy.add_execution(buy_fn, ['SPY'])
result = strategy.backtest(calc_bootstrap=False)
result.orders.head(10)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,SPY,2022-03-08,100,NaN,421.16,0.0
2,sell,SPY,2022-03-09,100,NaN,426.17,0.0
3,buy,SPY,2022-03-16,100,NaN,430.24,0.0
4,sell,SPY,2022-03-17,100,NaN,437.13,0.0
5,buy,SPY,2022-03-24,100,NaN,447.63,0.0
6,sell,SPY,2022-03-25,100,NaN,450.71,0.0
7,buy,SPY,2022-04-01,100,NaN,451.30,0.0
8,sell,SPY,2022-04-04,100,NaN,454.59,0.0
9,buy,SPY,2022-04-11,100,NaN,442.20,0.0


### ... cancel pending orders?

See the [cancel_pending_order](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.cancel_pending_order) and [cancel_all_pending_orders](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.cancel_all_pending_orders) methods.

In [3]:
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

def buy_fn(ctx: ExecContext):
    pending = tuple(ctx.pending_orders())
    if not pending and not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1
    if pending and ctx.close[-1] < 430:
        ctx.cancel_all_pending_orders(ctx.symbol)

config = StrategyConfig(buy_delay=5)
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023', config=config)
strategy.add_execution(buy_fn, ['SPY'])
result = strategy.backtest(calc_bootstrap=False)
result.orders.head(10)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:01 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:01


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,SPY,2022-03-23,100,NaN,446.10,0.0
2,sell,SPY,2022-03-24,100,NaN,447.63,0.0
3,buy,SPY,2022-03-31,100,NaN,454.96,0.0
4,sell,SPY,2022-04-01,100,NaN,451.30,0.0
5,buy,SPY,2022-04-08,100,NaN,448.29,0.0
6,sell,SPY,2022-04-11,100,NaN,442.20,0.0
7,buy,SPY,2022-04-19,100,NaN,441.74,0.0
8,sell,SPY,2022-04-20,100,NaN,445.53,0.0


### ... persist data across bars?

Use the [ExecContext#session](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.session) dictionary:

In [4]:
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

def buy_fn(ctx: ExecContext):
    if not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1
        order_count = ctx.session.get('order_count', 0)
        ctx.session['order_count'] = order_count + 1

strategy = Strategy(YFinance(), start_date='1/1/2022', end_date='1/1/2023')
strategy.add_execution(buy_fn, ['SPY'])
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2022-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-01-03 00:00:00 to 2022-12-30 00:00:00


100% (251 of 251) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00
